# About Pandas 

In [ ]:
Pandas: Python Data Analysis Library for data analysis and manipulation.
Pandas’ syntax is quite different from SQL. 
In Pandas, you apply operations on the dataset, and chain them, in order to transform and reshape the data the way you want it.


# load some data into Pandas

In [ ]:
import pandas as pd

airports = pd.read_csv('data/airports.csv')
airport_freq = pd.read_csv('data/airport-frequencies.csv')
runways = pd.read_csv('data/runways.csv')

SELECT, WHERE, DISTINCT, LIMIT

In [ ]:
Here are some SELECT statements.
We truncate results with LIMIT, and filter them with WHERE.
We use DISTINCT to remove duplicated results.

In [ ]:
#SQL:
select * from airports
select * from airports limit 3
select id from airports where ident = 'KLAX'
select distinct type from airport
#Pandas
airports
airports.head(3)
airports[airports.ident == 'KLAX'].id
airports.type.unique()

# SELECT with multiple conditions

In [ ]:
We join multiple conditions with an &. 

If we only want a subset of columns from the table, 
that subset is applied in another pair of square brackets.[]

In [ ]:
#SQL
select * from airports where iso_region = 'US-CA' and type = 'seaplane_base'
select ident, name, municipality from airports where iso_region = 'US-CA' and type = 'large_airport'
#Pandas
airports[(airports.iso_region == 'US-CA') & (airports.type == 'seaplane_base')]
airports[(airports.iso_region == 'US-CA') & (airports.type == 'large_airport')][['ident', 'name', 'municipality']]


# ORDER BY

In [ ]:
By default, Pandas will sort things in ascending order.
To reverse that, provide ascending=False.

In [ ]:
#SQL
select * from airport_freq where airport_ident = 'KLAX' order by type
select * from airport_freq where airport_ident = 'KLAX' order by type desc
#Pandas
airport_freq[airport_freq.airport_ident == 'KLAX'].sort_values('type')
airport_freq[airport_freq.airport_ident == 'KLAX'].sort_values('type', ascending=False)

# IN… NOT IN

In [ ]:
In pandas, .isin() operator works the same way. 
To negate any condition, use ~.

In [ ]:
#SQL
select * from airports where type in ('heliport', 'balloonport')
select * from airports where type not in ('heliport', 'balloonport')
#Pandas
airports[airports.type.isin(['heliport', 'balloonport'])]
airports[~airports.type.isin(['heliport', 'balloonport'])]

# GROUP BY, COUNT, ORDER BY

In [ ]:
Grouping is straightforward: use the .groupby() operator. 

In Pandas, .count() will return the number of non-null/NaN values.
To get the same result as the SQL COUNT, use .size().

In [ ]:
#SQL
select iso_country, type, count(*) from airports group by iso_country, type order by iso_country, type
select iso_country, type, count(*) from airports group by iso_country, type order by iso_country, count(*) desc
#Pandas
airports.groupby(['iso_country', 'type']).size()
airports.groupby(['iso_country', 'type']).size().to_frame('size').reset_index().sort_values(['iso_country', 'size'], ascending=[True, False])

In [ ]:
 .to_frame() 
Because we want to sort by our calculated field (size), this field needs to become part of the DataFrame. 
After grouping in Pandas, we get back a different type, called a GroupByObject. 
So we need to convert it back to a DataFrame.

 .reset_index()
 With .reset_index(), we restart row numbering for our data frame.

In [ ]:
Ex1: Pandas will sort things on the same list of fields by default, so there’s no need for a .sort_values() 
Ex2: If we want to use different fields for sorting, or DESC instead of ASC 

In [ ]:
#SQL
select iso_country, type, count(*) from airports group by iso_country, type order by iso_country, type
select iso_country, type, count(*) from airports group by iso_country, type order by iso_country, count(*) desc
#Pandas
airports.groupby(['iso_country', 'type']).size() #Ex1
airports.groupby(['iso_country', 'type']).size().to_frame('size').reset_index().sort_values(['iso_country', 'size'], ascending=[True, False]) #Ex2

# HAVING

In [ ]:
In SQL, you can additionally filter grouped data using a HAVING condition.

In Pandas, you can use .filter() and provide a Python function (or a lambda) 
that will return True if the group should be included into the result.

In [ ]:
#SQL
select type, count(*) from airports where iso_country = 'US' group by type having count(*) > 1000 order by count(*) desc
#Pandas
airports[airports.iso_country == 'US'].groupby('type').filter(lambda g: len(g) > 1000).groupby('type').size().sort_values(ascending=False)


# Top N records

In [ ]:
Ex1:We order things by airport_count and only select the top 10 countries with the largest count.
Ex2:we want “the next 10 after the top 10”

In [ ]:
#SQL
select iso_country from by_country order by size desc limit 10
select iso_country from by_country order by size desc limit 10 offset 10
#Pandas
by_country.nlargest(10, columns='airport_count') #Ex1
by_country.nlargest(20, columns='airport_count').tail(10) #Ex2

# Aggregate functions (MIN, MAX, MEAN)

In [ ]:
Calculate min, max, mean, and median length of a runway:

In [ ]:
#SQL
select max(length_ft), min(length_ft), avg(length_ft), median(length_ft) from runways
#Pandas
runways.agg({'length_ft': ['min', 'max', 'mean', 'median']})


In [ ]:
change Pandas from row to colume
simply transpose the dataframe with .T to get columns: df.T

# JOIN

In [ ]:
Use .merge() to join Pandas dataframes.
You need to provide which columns to join on (left_on and right_on)
and join type: inner (default), left (corresponds to LEFT OUTER in SQL), right (RIGHT OUTER), or outer (FULL OUTER).

In [ ]:
#SQL
select airport_ident, type, description, frequency_mhz from airport_freq join airports on airport_freq.airport_ref = airports.id where airports.ident = 'KLAX'
#
airport_freq.merge(airports[airports.ident == 'KLAX'][['id']], left_on='airport_ref', right_on='id', how='inner')[['airport_ident', 'type', 'description', 'frequency_mhz']]

# UNION ALL and UNION

In [ ]:
Use pd.concat() to UNION ALL two dataframes:
or
To deduplicate things (equivalent of UNION), you’d also have to add .drop_duplicates().

In [ ]:
#SQL
select name, municipality from airports where ident = 'KLAX' union all select name, municipality from airports where ident = 'KLGB'
#Pandas
pd.concat([airports[airports.ident == 'KLAX'][['name', 'municipality']], airports[airports.ident == 'KLGB'][['name', 'municipality']]])


# INSERT

In [ ]:
There’s no such thing as an INSERT in Pandas.
Instead, you would create a new dataframe containing new records, and then concat the two:

In [ ]:
#SQL
create table heroes (id integer, name text);
insert into heroes values (1, 'Harry Potter');
insert into heroes values (2, 'Ron Weasley');
insert into heroes values (3, 'Hermione Granger');
#Pandas
df1 = pd.DataFrame({'id': [1, 2], 'name': ['Harry Potter', 'Ron Weasley']})
df2 = pd.DataFrame({'id': [3], 'name': ['Hermione Granger']})
pd.concat([df1, df2]).reset_index(drop=True)

# UPDATE

In [ ]:
Now we need to fix some bad data in the original dataframe:

In [ ]:
#SQL
update airports set home_link = 'http://www.lawa.org/welcomelax.aspx' where ident == 'KLAX'
#Pandas
airports.loc[airports['ident'] == 'KLAX', 'home_link'] = 'http://www.lawa.org/welcomelax.aspx'

# DELETE

In [ ]:
Way 1: to “delete” things from a Pandas dataframe 
 subset the dataframe to rows you want to keep.

Way 2: get the indices of rows to delete, and .drop() rows using those indices:


In [ ]:
#SQL
delete from lax_freq where type = 'MISC'
#Pandas
lax_freq = lax_freq[lax_freq.type != 'MISC']  #Way 1
lax_freq.drop(lax_freq[lax_freq.type == 'MISC'].index)  #Way 2

# Immutability

In [ ]:
By default, most operators applied to a Pandas dataframe return a new object.
Some operators accept a parameter inplace=True, so you can work with the original dataframe instead. 

In [ ]:
#Reset an index in-place:
df.reset_index(drop=True, inplace=True)

#updated all values in a column:
df['url'] = 'http://google.com'

#added a new calculated column:
df['total_cost'] = df['price'] * df['quantity']

# More

In [ ]:
The nice thing about Pandas is that it’s more than just a query engine.
You can do other things with your data, such as:

In [ ]:
#Export to a multitude of formats:
df.to_csv(...)  # csv file
df.to_hdf(...)  # HDF5 file
df.to_pickle(...)  # serialized object
df.to_sql(...)  # to SQL database
df.to_excel(...)  # to Excel sheet
df.to_json(...)  # to JSON string
df.to_html(...)  # render as HTML table
df.to_feather(...)  # binary feather-format
df.to_latex(...)  # tabular environment table
df.to_stata(...)  # Stata binary data files
df.to_msgpack(...)	# msgpack (serialize) object
df.to_gbq(...)  # to a Google BigQuery table.
df.to_string(...)  # console-friendly tabular output.
df.to_clipboard(...) # clipboard that can be pasted into Excel

In [ ]:
#Plot it in graph
top_10.plot(
    x='iso_country', 
    y='airport_count',
    kind='barh',
    figsize=(10, 7),
    title='Top 10 countries with most airports')
